In [0]:
%sql

select *
from orders

In [0]:
%sql

--Gives each row a unique number based on the order_date.
SELECT 
  order_id, 
  order_date,
  ROW_NUMBER() OVER (ORDER BY order_date) AS row_num
FROM orders;


In [0]:
%sql
-- Real-Life Use Case: Show first product bought by each customer

SELECT 
c.first_name,od.product_name,od.order_id,
ROW_NUMBER() OVER(PARTITION BY c.customer_id ORDER BY o.order_date) AS PURCHASE_SEQUENCE
 FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id


---The First order is that where purchase_sequence=1

In [0]:
%sql
-- Real-Life Use Case: Show first product bought by each customer

SELECT 
c.first_name,od.product_name,od.order_id,
ROW_NUMBER() OVER(PARTITION BY c.customer_id ORDER BY o.order_date) AS PURCHASE_SEQUENCE
 FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id


---The First order is that where purchase_sequence=1

In [0]:
%sql
-- First product/order per customer using DISTINCT order_id
SELECT
    c.first_name,
    o.order_id,
    ROW_NUMBER() OVER (
        PARTITION BY c.customer_id 
        ORDER BY o.order_date
    ) AS purchase_sequence
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id


In [0]:
%sql
WITH ranked_orders AS (
    SELECT
        c.customer_id,
        c.first_name,
        o.order_id,
        ROW_NUMBER() OVER (
            PARTITION BY c.customer_id 
            ORDER BY o.order_date
        ) AS purchase_sequence
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
)
SELECT *
FROM ranked_orders
WHERE purchase_sequence = 1;


In [0]:
%sql
WITH ranked_orders AS (
    SELECT
        c.customer_id,
        c.first_name,
        o.order_id,
        RANK() OVER (
            PARTITION BY c.customer_id 
            ORDER BY o.order_date
        ) AS purchase_sequence
    FROM customers c
    JOIN orders o ON c.customer_id = o.customer_id
)
SELECT *
FROM ranked_orders
WHERE purchase_sequence = 1;


In [0]:
%sql
--RANK()
--Ranks rows by quantity; ties get same rank, but gaps are left in the sequence.

SELECT product_name,quantity,
RANK() OVER(ORDER BY quantity DESC) AS RANKING

 FROM order_details
     

In [0]:
%sql
-- Real-Life Use Case: Assign bonus based on best-selling products in each order

SELECT 
order_id,
product_name,
quantity,
RANK() OVER(PARTITION BY order_id ORDER BY quantity DESC) AS RANKING

 FROM order_details

In [0]:
%sql
--DENSE_RANK() -->RANK WITHOUT GAPS

SELECT 
  product_name, 
  quantity,
  DENSE_RANK() OVER (ORDER BY quantity DESC) AS DENSE_RANK,
  RANK() OVER (ORDER BY quantity DESC) AS RANK
FROM order_details;

In [0]:
%sql
--  Real-Life Use Case: Label product popularity tiers in clean rank buckets

SELECT 
product_name,
SUM(quantity) AS TOTAL_QUANTITY,
DENSE_RANK() OVER (ORDER BY SUM(quantity) DESC) AS POPULARITY_RANK
FROM order_details
GROUP BY 1

In [0]:
%sql
--LAG() - COMPARE WITH PREVIOUS ROWS

SELECT
order_id,order_date,
LAG(order_date) OVER(ORDER BY order_date) AS PREVIOUS_ORDER_DATE
 FROM ORDERS

In [0]:
%sql
-- Real-Life Use Case: Analyze customer s order trends (Did they buy more/less?)
SELECT 
  c.first_name, 
  o.order_id, 
  od.quantity,
  LAG(od.quantity) OVER (PARTITION BY c.customer_id ORDER BY o.order_date) AS previous_quantity
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id;
     


In [0]:
%sql
--LEAD() - COMPARE WITH NEXT ROWS

SELECT
order_id,order_date,
LEAD(order_date) OVER(ORDER BY order_date) AS NEXT_ORDER_DATE
 FROM ORDERS
     

In [0]:
%sql
--Real-Life Use Case: Predict what customers might buy next

SELECT 
  c.first_name, 
  od.product_name,
  LEAD(od.product_name) OVER (PARTITION BY c.customer_id ORDER BY o.order_date) AS possible_next_product
FROM customers c
JOIN orders o ON c.customer_id = o.customer_id
JOIN order_details od ON o.order_id = od.order_id;

In [0]:
%sql
--SUM() ---> RUNNING TOTAL OR Cummulative Sum 

select order_id,product_name,quantity,
SUM(quantity) OVER(ORDER BY order_id) AS CUMMULATIVE_SUM
from order_details